## Elastic net model - no maskers in features

For this section, we train an elastic net model for each fold of the cross-validation set and compute the corresponding metrics. The elastic net model minimises the loss function

$$L(w_1,\dots,w_n) = \sum_{i=1}^{n}\left(\left(y_i-w_ix_i\right)^2 + \alpha\left|w_i\right| + \beta w_i^2\right),$$

where $y_i$ are the ground truth labels, $x_i$ are the input features (or "predictor variables"), $w_i$ are the weights for the individual features (or "coefficients"), $n$ is the number of training samples, and $\alpha,\beta$ are regularisation parameters.

Note that the shapes of the feature arrays in the following block are:
- `X_train`: (20160, 132)
- `Y_train`: (20160,)
- `X_val`: (5040, 132)
- `Y_val`: (5040,)
- `X_test`: (48, 132)
- `Y_test`: (48,)

### Load
- Libraries
- Data files with responses

In [1]:
import sklearn.linear_model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

clip = lambda x, x_min = -1, x_max = 1: np.where(np.where(x < x_min,x_min,x) > x_max, x_max, np.where(x < x_min,x_min,x)) # Clip an array to values between x_min and x_max.

In [7]:
responses = pd.read_csv(os.path.join('..','data','responses_SoundLights.csv'), dtype = {'participant':str})
print(responses)

       participant  fold_r                          soundscape  \
0      ARAUS_00001      -1  R0091_segment_binaural_44100_1.wav   
1      ARAUS_00001       1  R0079_segment_binaural_44100_1.wav   
2      ARAUS_00001       1  R0056_segment_binaural_44100_2.wav   
3      ARAUS_00001       1  R0046_segment_binaural_44100_2.wav   
4      ARAUS_00001       1  R0092_segment_binaural_44100_1.wav   
...            ...     ...                                 ...   
27250  ARAUS_10005       0    R1007_segment_binaural_44100.wav   
27251  ARAUS_10005       0    R1006_segment_binaural_44100.wav   
27252  ARAUS_10005       0    R1008_segment_binaural_44100.wav   
27253  ARAUS_10005       0    R1007_segment_binaural_44100.wav   
27254  ARAUS_10005      -1  R0091_segment_binaural_44100_1.wav   

                       masker  smr  stimulus_index  wav_gain  time_taken  \
0           silence_00001.wav    0               1  0.000000      98.328   
1           silence_00001.wav    6               2  4.9

We first define the predictor variables that we will be using in `relevant_columns_enet`:

In [3]:
relevant_columns_enet = ['Savg_r','Smax_r','S05_r','S10_r','S20_r','S30_r','S40_r','S50_r','S60_r','S70_r','S80_r','S90_r','S95_r',
                         'Navg_r','Nmax_r','N05_r','N10_r','N20_r','N30_r','N40_r','N50_r','N60_r','N70_r','N80_r','N90_r','N95_r',
                         'Favg_r','Fmax_r','F05_r','F10_r','F20_r','F30_r','F40_r','F50_r','F60_r','F70_r','F80_r','F90_r','F95_r',
                         'LAavg_r','LAmin_r','LAmax_r','LA05_r','LA10_r','LA20_r','LA30_r','LA40_r','LA50_r','LA60_r','LA70_r','LA80_r','LA90_r','LA95_r',
                         'LCavg_r','LCmin_r','LCmax_r','LC05_r','LC10_r','LC20_r','LC30_r','LC40_r','LC50_r','LC60_r','LC70_r','LC80_r','LC90_r','LC95_r',
                         'Ravg_r','Rmax_r','R05_r','R10_r','R20_r','R30_r','R40_r','R50_r','R60_r','R70_r','R80_r','R90_r','R95_r',
                         'M00005_0_r','M00006_3_r','M00008_0_r','M00010_0_r','M00012_5_r','M00016_0_r','M00020_0_r','M00025_0_r','M00031_5_r','M00040_0_r',
                         'M00050_0_r','M00063_0_r','M00080_0_r','M00100_0_r','M00125_0_r','M00160_0_r','M00200_0_r','M00250_0_r','M00315_0_r','M00400_0_r',
                         'M00500_0_r','M00630_0_r','M00800_0_r','M01000_0_r','M01250_0_r','M01600_0_r','M02000_0_r','M02500_0_r','M03150_0_r','M04000_0_r',
                         'M05000_0_r','M06300_0_r','M08000_0_r','M10000_0_r','M12500_0_r','M16000_0_r','M20000_0_r']
#'Nrmc_r','Tgavg_r','Tavg_r','Tmax_r','T05_r','T10_r','T20_r','T30_r','T40_r','T50_r','T60_r','T70_r','T80_r','T90_r','T95_r',

In [11]:
model = sklearn.linear_model.ElasticNet
print(f'Investigating performance of {model()} model...')
MSEs_train = []
MSEs_val = []
MSEs_test = []

print('     |    Mean squared error    |         |       # samples      | #     | # NZ ')
print('Fold |--------+--------+--------| Inter-  |-------+-------+------| feat- | feat-')
print('     | Train  |   Val  |  Test  |  cept   | Train |  Val  | Test | ures  | ures ')
print('-----+--------+--------+--------+---------+-------+-------+------+-------+------')
for val_fold in [1,2,3,4,5]:

    # Extract dataframes
    df_train = responses[(responses['fold_r'] != val_fold) & (responses['fold_r'] > 0)] # For the training set, use all samples that are not in the test set (fold 0) and current validation fold.
    df_val   = responses[responses['fold_r'] == val_fold]
    df_test  = responses[responses['fold_r'] == 0].groupby(['soundscape','masker','smr']).mean() # For the test set, the same 48 stimuli were shown to all participants so we take the mean of their ratings as the ground truth
    
    # Get ground-truth labels
    Y_train = df_train['P_ground_truth'].values
    Y_val = df_val['P_ground_truth'].values
    Y_test = df_test['P_ground_truth'].values

    # Get features
    X_train = df_train[relevant_columns_enet].values 
    X_val = df_val[relevant_columns_enet].values
    X_test = df_test[relevant_columns_enet].values        

    # Fit model
    X_LR = model().fit(X_train, Y_train)


    # Get MSEs
    MSE_train = np.mean((X_LR.predict(X_train) - Y_train)**2)
    MSE_val = np.mean((X_LR.predict(X_val) - Y_val)**2)
    MSE_test = np.mean((X_LR.predict(X_test) - Y_test)**2)

    # Add metrics
    MSEs_train.append(MSE_train)
    MSEs_val.append(MSE_val)
    MSEs_test.append(MSE_test)

    print(f'{val_fold:4d} | {MSE_train:.4f} | {MSE_val:.4f} | {MSE_test:.4f} | {X_LR.intercept_:7.4f} | {X_train.shape[0]:5d} | {X_val.shape[0]:5d} | {X_test.shape[0]:^4d} | {X_train.shape[1]:^5d} | {np.sum(np.abs(X_LR.coef_) > 0):^5d} |')

print(f'Mean | {np.mean(MSEs_train):.4f} | {np.mean(MSEs_val):.4f} | {np.mean(MSEs_test):.4f} |')
print()

Investigating performance of ElasticNet() model...
     |    Mean squared error    |         |       # samples      | #     | # NZ 
Fold |--------+--------+--------| Inter-  |-------+-------+------| feat- | feat-
     | Train  |   Val  |  Test  |  cept   | Train |  Val  | Test | ures  | ures 
-----+--------+--------+--------+---------+-------+-------+------+-------+------
   1 | 0.1360 | 0.1321 | 0.0912 |  0.3074 | 20160 |  5040 |  48  |  117  |   3   |
   2 | 0.1368 | 0.1284 | 0.0954 |  0.3105 | 20160 |  5040 |  48  |  117  |   2   |
   3 | 0.1351 | 0.1384 | 0.0941 |  0.2898 | 20160 |  5040 |  48  |  117  |   2   |
   4 | 0.1329 | 0.1439 | 0.0941 |  0.3070 | 20160 |  5040 |  48  |  117  |   2   |
   5 | 0.1349 | 0.1355 | 0.0900 |  0.3029 | 20160 |  5040 |  48  |  117  |   3   |
Mean | 0.1351 | 0.1357 | 0.0930 |



Then, we generate the elastic net models and evaluate their performance:

### PLEASANTNESS

In [9]:
# Assuming X_LR is your linear regression model
coefficients = X_LR.coef_

# Pairing coefficients with feature names
feature_importances = list(zip(relevant_columns_enet, coefficients))

# Sorting feature importances by absolute value of coefficient
feature_importances.sort(key=lambda x: abs(x[1]), reverse=True)

# Displaying feature importances
for feature, importance in feature_importances:
    print(f"{feature}: {importance}")

Nmax_r: -0.007584002720763451
M10000_0_r: -0.0017722299435817064
M12500_0_r: -0.0007933806330985146
Savg_r: 0.0
Smax_r: 0.0
S05_r: 0.0
S10_r: 0.0
S20_r: 0.0
S30_r: -0.0
S40_r: -0.0
S50_r: -0.0
S60_r: -0.0
S70_r: -0.0
S80_r: 0.0
S90_r: 0.0
S95_r: 0.0
Navg_r: -0.0
Nrmc_r: -0.0
N05_r: -0.0
N10_r: -0.0
N20_r: -0.0
N30_r: -0.0
N40_r: -0.0
N50_r: -0.0
N60_r: -0.0
N70_r: -0.0
N80_r: -0.0
N90_r: -0.0
N95_r: -0.0
Favg_r: 0.0
Fmax_r: -0.0
F05_r: -0.0
F10_r: 0.0
F20_r: 0.0
F30_r: 0.0
F40_r: 0.0
F50_r: 0.0
F60_r: 0.0
F70_r: -0.0
F80_r: -0.0
F90_r: -0.0
F95_r: -0.0
LAavg_r: -0.0
LAmin_r: -0.0
LAmax_r: -0.0
LA05_r: -0.0
LA10_r: -0.0
LA20_r: -0.0
LA30_r: -0.0
LA40_r: -0.0
LA50_r: -0.0
LA60_r: -0.0
LA70_r: -0.0
LA80_r: -0.0
LA90_r: -0.0
LA95_r: -0.0
LCavg_r: -0.0
LCmin_r: -0.0
LCmax_r: -0.0
LC05_r: -0.0
LC10_r: -0.0
LC20_r: -0.0
LC30_r: -0.0
LC40_r: -0.0
LC50_r: -0.0
LC60_r: -0.0
LC70_r: -0.0
LC80_r: -0.0
LC90_r: -0.0
LC95_r: -0.0
Ravg_r: -0.0
Rmax_r: -0.0
R05_r: -0.0
R10_r: -0.0
R20_r: -0.0
R30_r: -0


We can see that only Nmax_r, M12500_0_r and M10000_0_r features play a role in the decision

### EVENTFULNESS

In [ ]:
model = sklearn.linear_model.ElasticNet
print(f'Investigating performance of {model()} model...')
MSEs_train = []
MSEs_val = []
MSEs_test = []

print('     |    Mean squared error    |         |       # samples      | #     | # NZ ')
print('Fold |--------+--------+--------| Inter-  |-------+-------+------| feat- | feat-')
print('     | Train  |   Val  |  Test  |  cept   | Train |  Val  | Test | ures  | ures ')
print('-----+--------+--------+--------+---------+-------+-------+------+-------+------')
for val_fold in [1,2,3,4,5]:

    # Extract dataframes
    df_train = responses[(responses['fold_r'] != val_fold) & (responses['fold_r'] > 0)] # For the training set, use all samples that are not in the test set (fold 0) and current validation fold.
    df_val   = responses[responses['fold_r'] == val_fold]
    df_test  = responses[responses['fold_r'] == 0].groupby(['soundscape','masker','smr']).mean() # For the test set, the same 48 stimuli were shown to all participants so we take the mean of their ratings as the ground truth

    # Get ground-truth labels
    Y_train = df_train['E_ground_truth'].values
    Y_val = df_val['E_ground_truth'].values
    Y_test = df_test['E_ground_truth'].values

    # Get features
    X_train = df_train[relevant_columns_enet].values 
    X_val = df_val[relevant_columns_enet].values
    X_test = df_test[relevant_columns_enet].values        

    # Fit model
    X_LR = model().fit(X_train, Y_train)

    # Get MSEs
    MSE_train = np.mean((X_LR.predict(X_train) - Y_train)**2)
    MSE_val = np.mean((X_LR.predict(X_val) - Y_val)**2)
    MSE_test = np.mean((X_LR.predict(X_test) - Y_test)**2)

    # Add metrics
    MSEs_train.append(MSE_train)
    MSEs_val.append(MSE_val)
    MSEs_test.append(MSE_test)

    print(f'{val_fold:4d} | {MSE_train:.4f} | {MSE_val:.4f} | {MSE_test:.4f} | {X_LR.intercept_:7.4f} | {X_train.shape[0]:5d} | {X_val.shape[0]:5d} | {X_test.shape[0]:^4d} | {X_train.shape[1]:^5d} | {np.sum(np.abs(X_LR.coef_) > 0):^5d}')

print(f'Mean | {np.mean(MSEs_train):.4f} | {np.mean(MSEs_val):.4f} | {np.mean(MSEs_test):.4f}')
print()

In [ ]:
# Assuming X_LR is your linear regression model
coefficients = X_LR.coef_

# Pairing coefficients with feature names
feature_importances = list(zip(relevant_columns_enet, coefficients))

# Sorting feature importances by absolute value of coefficient
feature_importances.sort(key=lambda x: abs(x[1]), reverse=True)

# Displaying feature importances
for feature, importance in feature_importances:
    print(f"{feature}: {importance}")

We can see that only Nmax_r, M00630_0_r, M00005_0_r and M00006_3_r features play a role in the decision

## Repeat process including maskers as input features

Which sound source is present in a soundscape can provide very usefull and direct information about the pleasantness and eventfulness of a sound scene. Therefore, we are introducing the masker column as an additional input feature to test any improvements in the predictions of the elastic models.

In [10]:
relevant_columns_enet = ['Savg_r','Smax_r','S05_r','S10_r','S20_r','S30_r','S40_r','S50_r','S60_r','S70_r','S80_r','S90_r','S95_r',
                         'Navg_r','Nrmc_r','Nmax_r','N05_r','N10_r','N20_r','N30_r','N40_r','N50_r','N60_r','N70_r','N80_r','N90_r','N95_r',
                         'Favg_r','Fmax_r','F05_r','F10_r','F20_r','F30_r','F40_r','F50_r','F60_r','F70_r','F80_r','F90_r','F95_r',
                         'LAavg_r','LAmin_r','LAmax_r','LA05_r','LA10_r','LA20_r','LA30_r','LA40_r','LA50_r','LA60_r','LA70_r','LA80_r','LA90_r','LA95_r',
                         'LCavg_r','LCmin_r','LCmax_r','LC05_r','LC10_r','LC20_r','LC30_r','LC40_r','LC50_r','LC60_r','LC70_r','LC80_r','LC90_r','LC95_r',
                         'Ravg_r','Rmax_r','R05_r','R10_r','R20_r','R30_r','R40_r','R50_r','R60_r','R70_r','R80_r','R90_r','R95_r',
                         'Tgavg_r','Tavg_r','Tmax_r','T05_r','T10_r','T20_r','T30_r','T40_r','T50_r','T60_r','T70_r','T80_r','T90_r','T95_r',
                         'M00005_0_r','M00006_3_r','M00008_0_r','M00010_0_r','M00012_5_r','M00016_0_r','M00020_0_r','M00025_0_r','M00031_5_r','M00040_0_r',
                         'M00050_0_r','M00063_0_r','M00080_0_r','M00100_0_r','M00125_0_r','M00160_0_r','M00200_0_r','M00250_0_r','M00315_0_r','M00400_0_r',
                         'M00500_0_r','M00630_0_r','M00800_0_r','M01000_0_r','M01250_0_r','M01600_0_r','M02000_0_r','M02500_0_r','M03150_0_r','M04000_0_r',
                         'M05000_0_r','M06300_0_r','M08000_0_r','M10000_0_r','M12500_0_r','M16000_0_r','M20000_0_r', 'masker_bird' , 'masker_construction',  
                         'masker_silence',  'masker_traffic','masker_water',  'masker_wind']

#### Pleasantness

In [12]:
model = sklearn.linear_model.ElasticNet
print(f'Investigating performance of {model()} model...')
MSEs_train = []
MSEs_val = []
MSEs_test = []

print('     |    Mean squared error    |         |       # samples      | #     | # NZ ')
print('Fold |--------+--------+--------| Inter-  |-------+-------+------| feat- | feat-')
print('     | Train  |   Val  |  Test  |  cept   | Train |  Val  | Test | ures  | ures ')
print('-----+--------+--------+--------+---------+-------+-------+------+-------+------')
for val_fold in [1,2,3,4,5]:

    # Extract dataframes
    df_train = responses[(responses['fold_r'] != val_fold) & (responses['fold_r'] > 0)] # For the training set, use all samples that are not in the test set (fold 0) and current validation fold.
    df_val   = responses[responses['fold_r'] == val_fold]
    df_test  = responses[responses['fold_r'] == 0].groupby(['soundscape','masker','smr']).mean() # For the test set, the same 48 stimuli were shown to all participants so we take the mean of their ratings as the ground truth

    # Get ground-truth labels
    Y_train = df_train['P_ground_truth'].values
    Y_val = df_val['P_ground_truth'].values
    Y_test = df_test['P_ground_truth'].values

    # Get features
    X_train = df_train[relevant_columns_enet].values 
    X_val = df_val[relevant_columns_enet].values
    X_test = df_test[relevant_columns_enet].values        

    # Fit model
    X_LR = model().fit(X_train, Y_train)

    # Get MSEs
    MSE_train = np.mean((X_LR.predict(X_train) - Y_train)**2)
    MSE_val = np.mean((X_LR.predict(X_val) - Y_val)**2)
    MSE_test = np.mean((X_LR.predict(X_test) - Y_test)**2)

    # Add metrics
    MSEs_train.append(MSE_train)
    MSEs_val.append(MSE_val)
    MSEs_test.append(MSE_test)

    print(f'{val_fold:4d} | {MSE_train:.4f} | {MSE_val:.4f} | {MSE_test:.4f} | {X_LR.intercept_:7.4f} | {X_train.shape[0]:5d} | {X_val.shape[0]:5d} | {X_test.shape[0]:^4d} | {X_train.shape[1]:^5d} | {np.sum(np.abs(X_LR.coef_) > 0):^5d}')

print(f'Mean | {np.mean(MSEs_train):.4f} | {np.mean(MSEs_val):.4f} | {np.mean(MSEs_test):.4f}')
print()

Investigating performance of ElasticNet() model...
     |    Mean squared error    |         |       # samples      | #     | # NZ 
Fold |--------+--------+--------| Inter-  |-------+-------+------| feat- | feat-
     | Train  |   Val  |  Test  |  cept   | Train |  Val  | Test | ures  | ures 
-----+--------+--------+--------+---------+-------+-------+------+-------+------
   1 | 0.1360 | 0.1321 | 0.0912 |  0.3074 | 20160 |  5040 |  48  |  138  |   3  
   2 | 0.1368 | 0.1284 | 0.0954 |  0.3105 | 20160 |  5040 |  48  |  138  |   2  
   3 | 0.1351 | 0.1384 | 0.0941 |  0.2898 | 20160 |  5040 |  48  |  138  |   2  
   4 | 0.1329 | 0.1439 | 0.0941 |  0.3070 | 20160 |  5040 |  48  |  138  |   2  
   5 | 0.1349 | 0.1355 | 0.0900 |  0.3029 | 20160 |  5040 |  48  |  138  |   3  
Mean | 0.1351 | 0.1357 | 0.0930



In [13]:
# Assuming X_LR is your linear regression model
coefficients = X_LR.coef_

# Pairing coefficients with feature names
feature_importances = list(zip(relevant_columns_enet, coefficients))

# Sorting feature importances by absolute value of coefficient
feature_importances.sort(key=lambda x: abs(x[1]), reverse=True)

# Displaying feature importances
for feature, importance in feature_importances:
    print(f"{feature}: {importance}")

Nmax_r: -0.008460668968782802
M12500_0_r: -0.0013485302212456784
M10000_0_r: -0.0004837700048420824
Savg_r: 0.0
Smax_r: 0.0
S05_r: 0.0
S10_r: 0.0
S20_r: -0.0
S30_r: -0.0
S40_r: -0.0
S50_r: -0.0
S60_r: -0.0
S70_r: -0.0
S80_r: 0.0
S90_r: 0.0
S95_r: 0.0
Navg_r: -0.0
Nrmc_r: -0.0
N05_r: -0.0
N10_r: -0.0
N20_r: -0.0
N30_r: -0.0
N40_r: -0.0
N50_r: -0.0
N60_r: -0.0
N70_r: -0.0
N80_r: -0.0
N90_r: -0.0
N95_r: -0.0
Favg_r: 0.0
Fmax_r: -0.0
F05_r: 0.0
F10_r: 0.0
F20_r: 0.0
F30_r: 0.0
F40_r: 0.0
F50_r: 0.0
F60_r: 0.0
F70_r: 0.0
F80_r: 0.0
F90_r: 0.0
F95_r: 0.0
LAavg_r: -0.0
LAmin_r: -0.0
LAmax_r: -0.0
LA05_r: -0.0
LA10_r: -0.0
LA20_r: -0.0
LA30_r: -0.0
LA40_r: -0.0
LA50_r: -0.0
LA60_r: -0.0
LA70_r: -0.0
LA80_r: -0.0
LA90_r: -0.0
LA95_r: -0.0
LCavg_r: -0.0
LCmin_r: -0.0
LCmax_r: -0.0
LC05_r: -0.0
LC10_r: -0.0
LC20_r: -0.0
LC30_r: -0.0
LC40_r: -0.0
LC50_r: -0.0
LC60_r: -0.0
LC70_r: -0.0
LC80_r: -0.0
LC90_r: -0.0
LC95_r: -0.0
Ravg_r: -0.0
Rmax_r: -0.0
R05_r: -0.0
R10_r: -0.0
R20_r: -0.0
R30_r: -0.0
R

#### Eventfulness

In [14]:
model = sklearn.linear_model.ElasticNet
print(f'Investigating performance of {model()} model...')
MSEs_train = []
MSEs_val = []
MSEs_test = []

print('     |    Mean squared error    |         |       # samples      | #     | # NZ ')
print('Fold |--------+--------+--------| Inter-  |-------+-------+------| feat- | feat-')
print('     | Train  |   Val  |  Test  |  cept   | Train |  Val  | Test | ures  | ures ')
print('-----+--------+--------+--------+---------+-------+-------+------+-------+------')
for val_fold in [1,2,3,4,5]:

    # Extract dataframes
    df_train = responses[(responses['fold_r'] != val_fold) & (responses['fold_r'] > 0)] # For the training set, use all samples that are not in the test set (fold 0) and current validation fold.
    df_val   = responses[responses['fold_r'] == val_fold]
    df_test  = responses[responses['fold_r'] == 0].groupby(['soundscape','masker','smr']).mean() # For the test set, the same 48 stimuli were shown to all participants so we take the mean of their ratings as the ground truth

    # Get ground-truth labels
    Y_train = df_train['E_ground_truth'].values
    Y_val = df_val['E_ground_truth'].values
    Y_test = df_test['E_ground_truth'].values

    # Get features
    X_train = df_train[relevant_columns_enet].values 
    X_val = df_val[relevant_columns_enet].values
    X_test = df_test[relevant_columns_enet].values        

    # Fit model
    X_LR = model().fit(X_train, Y_train)

    # Get MSEs
    MSE_train = np.mean((X_LR.predict(X_train) - Y_train)**2)
    MSE_val = np.mean((X_LR.predict(X_val) - Y_val)**2)
    MSE_test = np.mean((X_LR.predict(X_test) - Y_test)**2)

    # Add metrics
    MSEs_train.append(MSE_train)
    MSEs_val.append(MSE_val)
    MSEs_test.append(MSE_test)

    print(f'{val_fold:4d} | {MSE_train:.4f} | {MSE_val:.4f} | {MSE_test:.4f} | {X_LR.intercept_:7.4f} | {X_train.shape[0]:5d} | {X_val.shape[0]:5d} | {X_test.shape[0]:^4d} | {X_train.shape[1]:^5d} | {np.sum(np.abs(X_LR.coef_) > 0):^5d}')

print(f'Mean | {np.mean(MSEs_train):.4f} | {np.mean(MSEs_val):.4f} | {np.mean(MSEs_test):.4f}')
print()

Investigating performance of ElasticNet() model...
     |    Mean squared error    |         |       # samples      | #     | # NZ 
Fold |--------+--------+--------| Inter-  |-------+-------+------| feat- | feat-
     | Train  |   Val  |  Test  |  cept   | Train |  Val  | Test | ures  | ures 
-----+--------+--------+--------+---------+-------+-------+------+-------+------
   1 | 0.1376 | 0.1297 | 0.0425 | -0.2748 | 20160 |  5040 |  48  |  138  |   4  
   2 | 0.1373 | 0.1365 | 0.0463 | -0.2423 | 20160 |  5040 |  48  |  138  |   4  
   3 | 0.1390 | 0.1247 | 0.0434 | -0.3780 | 20160 |  5040 |  48  |  138  |   6  
   4 | 0.1352 | 0.1401 | 0.0439 | -0.3372 | 20160 |  5040 |  48  |  138  |   4  
   5 | 0.1322 | 0.1546 | 0.0450 | -0.3418 | 20160 |  5040 |  48  |  138  |   4  
Mean | 0.1363 | 0.1371 | 0.0442



In [15]:
# Assuming X_LR is your linear regression model
coefficients = X_LR.coef_

# Pairing coefficients with feature names
feature_importances = list(zip(relevant_columns_enet, coefficients))

# Sorting feature importances by absolute value of coefficient
feature_importances.sort(key=lambda x: abs(x[1]), reverse=True)

# Displaying feature importances
for feature, importance in feature_importances:
    print(f"{feature}: {importance}")

Nmax_r: 0.009261245154877234
M00630_0_r: 0.0018587244415044907
M00005_0_r: -0.0007956206191416479
M00006_3_r: -0.00020145418858936328
Savg_r: 0.0
Smax_r: 0.0
S05_r: 0.0
S10_r: 0.0
S20_r: 0.0
S30_r: 0.0
S40_r: 0.0
S50_r: 0.0
S60_r: 0.0
S70_r: 0.0
S80_r: 0.0
S90_r: 0.0
S95_r: 0.0
Navg_r: 0.0
Nrmc_r: 0.0
N05_r: 0.0
N10_r: 0.0
N20_r: 0.0
N30_r: 0.0
N40_r: 0.0
N50_r: 0.0
N60_r: 0.0
N70_r: 0.0
N80_r: 0.0
N90_r: 0.0
N95_r: 0.0
Favg_r: 0.0
Fmax_r: 0.0
F05_r: 0.0
F10_r: 0.0
F20_r: 0.0
F30_r: 0.0
F40_r: 0.0
F50_r: 0.0
F60_r: 0.0
F70_r: 0.0
F80_r: 0.0
F90_r: 0.0
F95_r: 0.0
LAavg_r: 0.0
LAmin_r: 0.0
LAmax_r: 0.0
LA05_r: 0.0
LA10_r: 0.0
LA20_r: 0.0
LA30_r: 0.0
LA40_r: 0.0
LA50_r: 0.0
LA60_r: 0.0
LA70_r: 0.0
LA80_r: 0.0
LA90_r: 0.0
LA95_r: 0.0
LCavg_r: 0.0
LCmin_r: 0.0
LCmax_r: 0.0
LC05_r: 0.0
LC10_r: 0.0
LC20_r: 0.0
LC30_r: 0.0
LC40_r: 0.0
LC50_r: 0.0
LC60_r: 0.0
LC70_r: 0.0
LC80_r: 0.0
LC90_r: 0.0
LC95_r: 0.0
Ravg_r: 0.0
Rmax_r: 0.0
R05_r: 0.0
R10_r: 0.0
R20_r: 0.0
R30_r: 0.0
R40_r: 0.0
R50_r: 0.0

Adding the maskers as features didn't provide any improvements to the elastic net linear regression model. In fact, we can see which features are employed or perform a more important role are only :

For pleasantness
- Nmax_r: -0.008460668968782803
- M12500_0_r: -0.00134853022124568
- M10000_0_r: -0.0004837700048420805

For eventfulness
- Nmax_r: 0.009261245154877232
- M00630_0_r: 0.001858724441504498
- M00005_0_r: -0.0007956206191416494
- M00006_3_r: -0.00020145418858936152

Meaning that only features related to loudness (Nmax_r) and to frequency energy distribution (M...) play a role in the model. The coefficients indicate the contribution of each feature to the model's predictions. Here's how to interpret the results:
- Positive Coefficient: A positive coefficient (e.g., Savg_r: 0.123) means that as the value of the feature increases, the predicted target variable is expected to increase by the coefficient value.
- Negative Coefficient: A negative coefficient (e.g., Nmax_r: -0.008) means that as the value of the feature increases, the predicted target variable is expected to decrease by the absolute value of the coefficient.
- Coefficient Magnitude: The magnitude (absolute value) of the coefficient indicates the strength of the effect. Larger magnitudes imply a stronger impact on the predictions.

Then, it makes sense that as loudness increases, pleasantness decreases but eventfulness increases. It also makes sense that pleasantness is more influenced by high frequency bands content (they can be more annoying) and eventfulness with low frequency content (?why).